### Instruções para Carregamento e Ajustes dos Dados
- Certifique-se de que seus dados incluam o tempo de entrega para cada entrega.
- A coluna Proporcao_Entrega_Em_Tempo deve ser baseada na comparação entre o tempo de entrega e a meta de 30 minutos.
- Criar uma nova coluna que indica se a entrega foi realizada dentro de 30 minutos: Calcule a proporção de entregas realizadas dentro de 30 minutos, que será sua variável preditora.
- Carregue seus dados como "df" e faça os devidos ajustes para garantir que eles estejam no formato esperado para o modelo.
- A coluna 'Proporcao_Entrega_Em_Tempo' foi utilizada como variável preditora no exemplo. No código, um DataFrame fictício foi gerado; não se esqueça de substituir com os dados reais antes de rodar o modelo.
- Meta = 75% das entregas abaixo de 30 minutos #Ajuste conforme necessário.

## Proposta para Uso no Dia-a-Dia
A solução será aplicada para prever o desempenho do processo de entrega ao longo do mês, fornecendo estimativas contínuas e ajustáveis. O objetivo é obter uma previsão confiável sobre o percentual de entregas realizadas em menos de 30 minutos, permitindo ações corretivas proativas, se necessário.

### Funcionamento da Solução
Ao longo do mês o modelo preverá a proporção de entregas realizadas abaixo de 30 minutos ao final do mês (prevendo o valor final do processo no mês).
Meta: 75% ou mais dos lotes entregues no prazo.

Ações baseadas na previsão: 
- Se % ≥ 75%: Nenhuma ação corretiva necessária.
- Se % < 75%: Aplicar o modelo GBM para identificar os fatores que causam atrasos e tomar medidas específicas.

### Estratégia de Modelagem

#### Modelo Bayesiano como Forecast Dinâmico
- O modelo Bayesiano será utilizado para calcular a probabilidade mensal de a % de entregas abaixo de 30 minutos estar abaixo da meta. 
- As previsões serão ajustadas continuamente à medida que novos dados de entrega são adicionados diariamente.
- Número de amostras: No uso de um modelo Bayesiano, a quantidade de dados históricos é crucial para obter estimativas precisas dos parâmetros (μ e σ). Idealmente, você deve ter pelo menos 30 a 50 pontos de dados para garantir que as distribuições posteriores sejam confiáveis e bem definidas.
- Tendência e variação: Quanto mais dados você tiver, mais facilmente o modelo poderá capturar as tendências e variações sazonais no processo de entrega, proporcionando previsões mais estáveis e precisas ao longo do tempo.
- Distribuição Log-Normal: O modelo considera a distribuição log-normal, que é a distribuição esperada para processos que envolvem tempos de entrega, como em casos urgentes de medicamentos (por exemplo, pacientes com dor). A distribuição log-normal é ideal para representar esses tempos, que geralmente possuem uma cauda mais pesada e são assimétricos.

#### Integração com Gradient Boosting Machines (GBM)
A previsão diária gerada pelo modelo Bayesiano indicará a necessidade de uma análise mais apronfundada. 
O GBM será usado para refinar a previsão e identificar as principais causas de atrasos.
Features usadas no GBM: ['Setor prescrição', 'Cd material', 'Usuário entrega', 'Dia', 'Hora de geração (min)', 'Hora de Entrega (min)'].

### Análise e Ação
A combinação dos modelos permite:
- Previsão Precisa do Desempenho: O modelo Bayesiano fornece uma estimativa inicial confiável.
- Identificação das Causas: O GBM identifica as variáveis mais influentes nos atrasos e auxilia na formulação de estratégias de melhoria.

### Vantagens do Modelo Proposto
- Previsão Contínua: A abordagem Bayesiana permite atualizações diárias, garantindo previsões mais precisas à medida que o mês avança.
- Insights Aprofundados: O GBM complementa a análise, ajudando a diagnosticar problemas específicos no processo.
- Tomada de Decisão Proativa: Com base na previsão, a equipe pode atuar antes que a meta de 75% seja comprometida.

Essa solução combina a adaptabilidade do modelo Bayesiano com o poder analítico do GBM, garantindo previsões confiáveis e ações corretivas orientadas por dados.

In [ ]:
import pymc3 as pm
import numpy as np
import pandas as pd

# Gerando dados de 1 ano de entregas (365 dias)
# Gerar uma série de datas para 1 ano a partir de '2024-12-03'
dates = pd.date_range(start='2024-12-03', periods=365, freq='D')

# Gerar valores aleatórios para 'Proporcao_Entrega_Em_Tempo' (substitua com dados reais)
# Exemplo: valores entre 0 e 1, representando a proporção de entregas dentro do prazo
proporcao_entrega = np.random.rand(365)

# Criar o DataFrame com os dados gerados
df = pd.DataFrame({
    'Data': dates,
    'Proporcao_Entrega_Em_Tempo': proporcao_entrega
})

# Extraindo a coluna 'Proporcao_Entrega_Em_Tempo' para treinamento do modelo
y_train = df['Proporcao_Entrega_Em_Tempo'].values

# Modelo Bayesiano com Distribuição Lognormal
with pm.Model() as bayesian_model:
    # Priors para a média e desvio padrão logarítmicos
    mu = pm.Normal("mu", mu=np.log(y_train).mean(), sigma=np.log(y_train).std())
    sigma = pm.HalfNormal("sigma", sigma=np.log(y_train).std())

    # Likelihood usando uma LogNormal
    likelihood = pm.Lognormal("likelihood", mu=mu, sigma=sigma, observed=y_train)

    # Amostragem para posterior
    bayesian_trace = pm.sample(1000, tune=1000, return_inferencedata=False, chains=2, progressbar=True)

# Posterior da média e desvio padrão logarítmicos
posterior_mu = bayesian_trace["mu"].mean()
posterior_sigma = bayesian_trace["sigma"].mean()
print(f"Posterior da média logarítmica: {posterior_mu:.4f}, Posterior do desvio padrão logarítmico: {posterior_sigma:.4f}")

# Função para gerar previsão baseada no modelo posterior
def predict_proportion(mu, sigma, num_samples=1000):
    # Gerar previsões com base no posterior (usando LogNormal)
    return np.random.lognormal(mu, sigma, num_samples)

# Gerar previsões da proporção de entregas dentro do prazo
predictions = predict_proportion(posterior_mu, posterior_sigma, num_samples=1000)

# Mostrar algumas previsões
print("Previsões para a proporção de entregas dentro do prazo:")
print(predictions[:10])

# Calcular a probabilidade de que a proporção de entregas fique abaixo da meta de 75%
prob_below_75 = np.mean(predictions < 0.75)
print(f"Probabilidade de a proporção de entregas ficar abaixo de 75%: {prob_below_75:.4f}")